<a href="https://colab.research.google.com/github/RajMV05102004/DeepLeanring/blob/MiniProjects/WikiWordPredictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

# Read the file assuming it's space-separated or tab-separated
df = pd.read_csv(r"/content/wiki.train.tokens", sep="\t", header=None, names=["Text"])

# Display the first few lines


In [29]:
valid_df=pd.read_csv(r"/content/wiki.valid.tokens", sep="\t", header=None, names=["Text"])

In [2]:
df.sample(10)

,Text
2893,"In 1848 , the influential Gothic Revival arch..."
2221,= = Analysis = =
3623,"Photographs recur as a motif , such as family..."
2720,The works Etty painted following his return b...
1152,"On September 25 , 2001 , Jordan announced his..."
3936,"GameZone praised the game 's sound effects , ..."
2886,= = = <unk> = = =
1272,The Indian Navy 's Advanced Technology Vessel...
730,= = The compromise = =
3734,"The book was "" highly commended "" and runner ..."


In [3]:
#Replacing unk with empty string
df["Text"] = df["Text"].str.replace("<unk>", "")


In [4]:
# Remove all special characters using regex
df["Text"] = df["Text"].str.replace(r"[^a-zA-Z\s]", "", regex=True)


In [5]:
#Convert everything to lowercase
df['Text']=df['Text'].str.lower()

In [6]:
df.head(5)

,Text
0,valkyria chronicles iii
1,senj no valkyria chronicles japanese l...
2,the game began development in carrying over...
3,it met with positive sales in japan and was ...
4,gameplay


In [7]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

In [8]:
tokenizer=Tokenizer()

In [9]:
def fit_tokenizer(obj,tokenizer,col):
  tokenizer.fit_on_texts(obj[col])


In [10]:
fit_tokenizer(df,tokenizer,'Text')

In [11]:
tokenizer.word_index

{'the': 1,
 'of': 2,
 'and': 3,
 'in': 4,
 'to': 5,
 'a': 6,
 'was': 7,
 's': 8,
 'on': 9,
 'as': 10,
 'for': 11,
 'that': 12,
 'with': 13,
 'by': 14,
 'at': 15,
 'he': 16,
 'his': 17,
 'is': 18,
 'it': 19,
 'from': 20,
 'were': 21,
 'an': 22,
 'had': 23,
 'which': 24,
 'be': 25,
 'this': 26,
 'her': 27,
 'their': 28,
 'not': 29,
 'but': 30,
 'she': 31,
 'are': 32,
 'after': 33,
 'also': 34,
 'one': 35,
 'or': 36,
 'first': 37,
 'they': 38,
 'its': 39,
 'two': 40,
 'new': 41,
 'who': 42,
 'has': 43,
 'have': 44,
 'would': 45,
 'been': 46,
 'time': 47,
 'during': 48,
 'all': 49,
 'when': 50,
 'other': 51,
 'more': 52,
 'into': 53,
 'over': 54,
 'only': 55,
 'i': 56,
 'while': 57,
 'game': 58,
 'him': 59,
 'later': 60,
 'made': 61,
 'about': 62,
 'up': 63,
 'season': 64,
 'some': 65,
 'year': 66,
 'album': 67,
 'out': 68,
 'most': 69,
 'three': 70,
 'these': 71,
 'no': 72,
 'wall': 73,
 'british': 74,
 'home': 75,
 'there': 76,
 'under': 77,
 'such': 78,
 'may': 79,
 'film': 80,
 'than':

In [12]:
tokenizer.word_counts

OrderedDict([('valkyria', 54),
             ('chronicles', 41),
             ('iii', 113),
             ('senj', 5),
             ('no', 339),
             ('japanese', 25),
             ('lit', 9),
             ('of', 10953),
             ('the', 25289),
             ('battlefield', 9),
             ('commonly', 12),
             ('referred', 33),
             ('to', 8051),
             ('as', 2871),
             ('outside', 65),
             ('japan', 35),
             ('is', 1804),
             ('a', 7126),
             ('tactical', 4),
             ('role', 135),
             ('playing', 57),
             ('video', 154),
             ('game', 389),
             ('developed', 89),
             ('by', 2510),
             ('sega', 5),
             ('and', 9726),
             ('mediavision', 3),
             ('for', 2870),
             ('playstation', 9),
             ('portable', 5),
             ('released', 194),
             ('in', 8486),
             ('january', 113),
            

In [13]:
len(tokenizer.word_index)

23041

In [14]:
data=df.copy()# Storing the original DataFrame

In [15]:
Dataset=pd.DataFrame();

In [16]:
#We are creating a dataset where a sequence of words are stored in an non-decreasing manner
input_sequence=[]

for sentence in df["Text"]:
  tokennized_sentence=tokenizer.texts_to_sequences([sentence])[0]

  for i in range(1,len(tokennized_sentence)):
    n_gram=tokennized_sentence[:i+1]
    input_sequence.append(n_gram)

In [17]:
input_sequence[:3]

[[715, 1013], [715, 1013, 296], [5924, 72]]

In [18]:
#We need the maximum length in the input sequence
maxlen=max(len(x) for x in input_sequence)

In [19]:
maxlen# This is the  maximum size of the input sequence

461

In [20]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [21]:
#now we will pad the input sequences to the maxlen
padded_input_sequence=pad_sequences(input_sequence,maxlen=maxlen,padding='pre')

In [30]:
import pickle

In [31]:
with open('padded_input_sequence.pkl', 'wb') as f:
    pickle.dump((padded_input_sequence, tokenizer), f)

In [22]:
X=padded_input_sequence[:,:-1]
y=padded_input_sequence[:,-1]
#y=np.expand_dims(y,axis=1)

In [23]:
from tensorflow.keras.utils import to_categorical
#Applying categorical transformation to make it a multiclass classification problem
y=to_categorical(y,num_classes=len(tokenizer.word_index)+1)

Model1 Structure:
1. Embedding Layer
2. Bidirectional LSTM layer
3. Bidirectional LSTM layer
4. Dense Layer

In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,Bidirectional,LSTM,Dense

In [27]:
model1=Sequential()
model1.add(Embedding(input_dim=len(tokenizer.word_index)+1,output_dim=200))
'''
output_dim: This is the size of the word vectors (embeddings). You're setting it to 200, meaning each word will be represented by a 200-dimensional vector.
This layer converts each word index (from the tokenizer) into a dense embedding vector
'''
model1.add(Bidirectional(LSTM(256,return_sequences=True)))
model1.add(Bidirectional(LSTM(256)))
model1.add(Dense(len(tokenizer.word_index)+1,activation='softmax'))

Summary of the Model
Input: Integer-encoded sequences of words (from the tokenizer).

Embedding Layer: Converts words into dense 200-dimensional vectors.

Bidirectional LSTMs: Two layers of bidirectional LSTMs process the sequence to capture contextual information.

Output Layer: A dense layer with softmax activation predicts the next word (or class) based on the processed sequence.

In [28]:
model1.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(X,y,epochs=10,batch_size=128)